In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import utils #Stores info like UvA's ISATCODES

In [2]:
YEAR = 2025
df_SL = pd.read_csv("data/raw/SL_21PK_2025_V37_20250616_sleutelbestand_pk.csv", sep=";")


In [ ]:
typestudentmap= {
    1:"nieuw_in_hoger_onderwijs",
    2:"hbo_instroom",
    4:"universiteit_switcher",
    5:"studie_switcher",
    6:"herinschrijver",
}
df_SL["StudentType"] = df_SL["1cHO_K"].map(typestudentmap)

def StudentTypeExtra(row):

    # if isinstance(row["MSc_MQP"],float) == False: # of het nan is
    #     return "MQP"

    if row["AcademicLevel"] == "M" and row["StudentType"] =="studie_switcher" and row["1cHO_L"] == 3:
        return "uva_bachelor_student"
    
    if row["AcademicLevel"] == "M" and row["StudentType"] =="studie_switcher" and row["1cHO_L"] == 4:
        #Niet nieuw bij de 
        return "uva_master_student"
                
    if row["AcademicLevel"] == "M" and row["StudentType"] =="studie_switcher" and row["1cHO_L"] == 5:
        return "uva_master_student"

    if row["AcademicLevel"] == "B" and row["StudentType"] =="studie_switcher" and row["1cHO_L"] == 3:
        return "uva_master_student" 

    if row["AcademicLevel"] == "B" and row["StudentType"] =="studie_switcher" and row["1cHO_L"] == 4:
        return "uva_bachelor_student" 
    
    if row["AcademicLevel"] == "B" and row["StudentType"] =="studie_switcher" and row["1cHO_L"] == 5:
        return "uva_bachelor_student" 
    

    if row["AcademicLevel"] == "M" and row["StudentType"] =="universiteit_switcher" and row["1cHO_L"] == 3:
        return "nietuva_bachelor_student"
    
    if row["AcademicLevel"] == "M" and row["StudentType"] =="universiteit_switcher" and row["1cHO_L"] == 4:
        return "nietuva_master_student"
    
    
    if row["AcademicLevel"] == "B" and row["StudentType"] =="universiteit_switcher" and row["1cHO_L"] == 3:
        #Maarten dit klopt. Want roel kent blijkbaar 3 mensen. Er zijn studenten die na een master zich aan melden voor een bachelor.
        return "nietuva_master_student"
        
    if row["AcademicLevel"] == "B" and row["StudentType"] =="universiteit_switcher" and row["1cHO_L"] == 4:
        return "nietuva_bachelor_student"


    #hier onder alleen nog maar nieuw in hoger onderwijs
    if row["StudentType"] !="nieuw_in_hoger_onderwijs":
        return row["StudentType"]

    if row["Herkomst"] == "N":
        return "NL_nieuw_in_hoger_onderwijs"

    if row["Herkomst"] == "E":
        return "EER_nieuw_in_hoger_onderwijs"

    if row["Herkomst"] == "R":
        return "rest_nieuw_in_hoger_onderwijs"

    if row["Herkomst"] == "O":
        return "rest_nieuw_in_hoger_onderwijs"
            
    return row["StudentType"]

df_SL["StudentType"] = df_SL.apply(lambda x: StudentTypeExtra(x), axis=1)

In [ ]:
#indien een student master als bachelor aanmeldingen heeft
def ChooseStudentType(studentTypes):
    _studentTypes = set(studentTypes)
    _studentTypes.discard("herinschrijver")
    
    if len(_studentTypes)==0:
        return "herinschrijver"


    if len(_studentTypes)==1:
        return list(_studentTypes)[0]


    if('nietuva_master_student' in _studentTypes):
        return "nietuva_master_student"
    
    if('uva_master_student' in _studentTypes):
        return "uva_master_student"    
    
    return _studentTypes

df2 = df_SL.groupby(["Studielinknummer","Geslacht","Herkomst","Meercode_V","Meercode_A"]).agg({
    "Isatcode": lambda x: list(x),
    "Maand": lambda x: list(x),
    "Status": lambda x: list(x),
    "AcademicLevel": lambda x: list(x),
    "StudentType": lambda x: ChooseStudentType(x),    
    "Hogerejaars": lambda x: list(x),
    "Fixus": lambda x: list(x),
    "Herinschrijving": lambda x: list(x),
    "1cHO_L": lambda x: list(x),
    "1cHO_K": lambda x: list(x),    
}).reset_index()

In [6]:
#Feature enginerings
def amount_of_applied_multiple_FEB_programs(row) -> int:
    
    #Dit is een set omdat soms dubbele SL aanmeldingen zijn.
    return sum(code in set(row["Isatcode"]) for code in utils.ISATCODES)


def also_looking_elsewhere(row) -> str:
    SL_applications = row["Meercode_V"] + row["Meercode_A"]

    amountof_other_NL_instituition = SL_applications - len(row["Isatcode"])


    #Entrenaurship
    if(65006 in row["Isatcode"]) and amountof_other_NL_instituition >0:
        amountof_other_NL_instituition -= 1


    #Andere instelling wellicht herinschrijving
    if(4 in row["1cHO_K"]):
        amountof_other_NL_instituition -= 1


    return amountof_other_NL_instituition > 0
    
df2["amount_of_applied_multiple_FEB_programs"] = df2.apply(lambda x: amount_of_applied_multiple_FEB_programs(x), axis=1).astype(int)
df2["also_looking_elsewhere"] = df2.apply(lambda x: also_looking_elsewhere(x), axis=1).astype(int)

In [7]:
df2[["Studielinknummer","StudentType","Geslacht","Herkomst","Isatcode","Status","amount_of_applied_multiple_FEB_programs","also_looking_elsewhere"]].to_excel(f"Sleutelbestanden_{YEAR}.xlsx",index=False)